# IOC catalog

In [ ]:
import pandas as pd
import numpy as np
import requests

### Read html with Pandas

In [ ]:
ioc = pd.read_html('http://www.ioc-sealevelmonitoring.org/ssc/')[0]

In [ ]:
ioc.drop(ioc.tail(1).index,inplace=True)

In [ ]:
ioc

In [ ]:
#save DataFrame
ioc.to_csv('ioc.csv',encoding='utf-8',index=False)

## Additional IOC table with real time info 

In [ ]:
import pandas as pd
ioc2 = pd.read_html('http://www.ioc-sealevelmonitoring.org/list.php?operator=&showall=all&output=general#')

In [ ]:
ioc2 = ioc2[6].drop(0)
nh = ioc2.iloc[0]
ioc2 = ioc2[1:]
ioc2.columns = nh.values
ioc2 = ioc2.iloc[:,:10]
ioc2.reset_index(inplace=True, drop=True)

ioc2 = ioc2.drop(ioc2.columns[6:7],axis=1)

In [ ]:
ioc2

In [ ]:
#save DataFrame
ioc2.to_csv('ioc2.csv',encoding='utf-8',index=False)

### merge

In [ ]:
#Merge the two ioc DataFrames
ioc3 = ioc.merge(ioc2, left_on=['Station Name'], right_on=['Location'])

In [ ]:
ioc3 = ioc3.drop(['Details','Edit','Connection','DCP ID','Delay','TransmitInterval'], axis=1)

In [ ]:
#save DataFrame
ioc3.to_csv('ioc3.csv',encoding='utf-8',index=False)

In [ ]:
ioc3

### Find tgs within a lat/lon window

In [ ]:
minlat = 17.,
maxlat = 25.,
minlon = 100.0,
maxlon = 130.0,

In [ ]:
w = ioc.loc[(ioc['Longitude'] > minlon) & (ioc['Longitude'] < maxlon) & (ioc['Latitude'] > minlat) & (ioc['Latitude'] < maxlat)]

In [ ]:
w.reset_index(inplace=True, drop=True)

In [ ]:
w

## get the data

In [ ]:
url = "http://www.ioc-sealevelmonitoring.org/bgraph.php?code={}&output=tab&period=0.5&endtime={}".format('quar','2020-2-1') # use IOC code value
url

In [ ]:
data = pd.read_html(url, header=0)[0]
data